In [0]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
#!wget -q http://apache.osuosl.org/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.6.tgz

In [0]:
#!tar xf spark-2.3.3-bin-hadoop2.6.tgz

In [0]:
#!pip install -q pyspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.6"

In [0]:
#import findspark
#findspark.init()
#from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession 

In [0]:
sc = SparkContext("local", "Simple App")
spark = SparkSession.builder.getOrCreate()

In [0]:
#from google.colab import files
#files.upload()

In [0]:
!ls

ratings_small.csv  spark-2.3.3-bin-hadoop2.6	  spark-warehouse
sample_data	   spark-2.3.3-bin-hadoop2.6.tgz


In [0]:
#rm -r pyspark-2.3.3
#rm spark-2.3.3-bin-hadoop2.7.tgz.1

In [0]:
from pyspark.sql.functions import col, min, avg

#1. Import the dataset and perform some basic calculations

a. First we upload the file using pyspark

In [0]:
ratings_small = spark.read.csv('ratings_small.csv',inferSchema=True, header =True)

b. Checking the number of columns in the file

In [0]:
ratings_small.columns

['userId', 'movieId', 'rating', 'timestamp']

c. Viewing first 20 rows of the data

In [0]:
ratings_small.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows



d. Calculation the number of distinct users and movies.

In [0]:
user_count=ratings_small.select("userId").distinct().count()
print("Number of users: ", user_count)

movie_count=ratings_small.select("movieId").distinct().count()
print("Number of movies: ", movie_count)


Number of users:  671
Number of movies:  9066


e. The GroupBy and Filter Methods

In [0]:
# View the ratings dataset
ratings_small.show()

# Filter out all userIds greater than 100
ratings_small.filter(col("userId") < 100).show()

# Group data by userId, count ratings
ratings_small.groupBy("userId").count().show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|126075

# Summary Statistics

In [0]:
# Min num ratings for movies
print("Movie with the fewest ratings: ")
ratings_small.groupBy("movieId").count().select(min("count")).show()

# Avg num ratings per movie
print("Avg num ratings per movie: ")
ratings_small.groupBy("movieId").count().select(avg("count")).show()

# Min num ratings for user
print("User with the fewest ratings: ")
ratings_small.groupBy("userId").count().select(min("count")).show()

# Avg num ratings per users
print("Avg num ratings per user: ")
ratings_small.groupBy("userId").count().select(avg("count")).show()

Movie with the fewest ratings: 
+----------+
|min(count)|
+----------+
|         1|
+----------+

Avg num ratings per movie: 
+------------------+
|        avg(count)|
+------------------+
|11.030664019413193|
+------------------+

User with the fewest ratings: 
+----------+
|min(count)|
+----------+
|        20|
+----------+

Avg num ratings per user: 
+------------------+
|        avg(count)|
+------------------+
|149.03725782414307|
+------------------+



# Calculate Sparcity of the DataFrame

In [0]:
# Count the total number of ratings in the dataset
numerator = ratings_small.select("rating").count()

# Count the number of distinct userIds and distinct movieIds.
num_users = ratings_small.select("userId").distinct().count()
num_movies = ratings_small.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print ("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.36% empty.


# View Schema

In [0]:
# Use the .printSchema() method to see the datatypes of the ratings dataset.
ratings_small.printSchema()

# Tell Spark to convert the columns to the proper data types.
ratings_small = ratings_small.select(ratings_small.userId.cast("integer"), ratings_small.movieId.cast("integer"), ratings_small.rating.cast("double"))

# Call .printSchema() again to confirm the columns are now in the correct format
ratings_small.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



# Create test/train splits and build your ALS model

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [0]:
# Create test and train set
(train, test) = ratings_small.randomSplit([0.80, 0.20], seed = 1234)
ratings_small.show()
# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", rank =10, maxIter =15, regParam =.1,
            coldStartStrategy="drop", nonnegative =True, implicitPrefs = False)

# Fit the mdoel to the training_data
model = als.fit(train)

# Generate predictions on the test_data
test_predictions = model.transform(test)
test_predictions.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     31|   2.5|
|     1|   1029|   3.0|
|     1|   1061|   3.0|
|     1|   1129|   2.0|
|     1|   1172|   4.0|
|     1|   1263|   2.0|
|     1|   1287|   2.0|
|     1|   1293|   2.0|
|     1|   1339|   3.5|
|     1|   1343|   2.0|
|     1|   1371|   2.5|
|     1|   1405|   1.0|
|     1|   1953|   4.0|
|     1|   2105|   4.0|
|     1|   2150|   3.0|
|     1|   2193|   2.0|
|     1|   2294|   2.0|
|     1|   2455|   2.5|
|     1|   2968|   1.0|
|     1|   3671|   3.0|
+------+-------+------+
only showing top 20 rows

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   232|    463|   4.0| 3.4895976|
|   452|    463|   2.0| 2.8818955|
|   534|    463|   4.0|   3.36804|
|    86|    471|   4.0| 4.1151204|
|   292|    471|   3.5| 4.0617824|
|   306|    471|   3.0| 3.7516801|
|   491|    471|   3.0| 4.0695353|
|   309|    471|   4.0|  4.269315|
|   607|    4

In [0]:
# Create test and train set

#(train, test) = ratings_small.randomSplit([0.80, 0.20], seed = 1234)
#ratings_small.show()

# Create ALS model

#als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False)

# Confirm that a model called "als" was created

#type(als)

In [0]:
# Add hyperparameters and their respective values to param_grid

#param_grid = ParamGridBuilder() \
#            .addGrid(als.rank, [10, 50, 100, 150]) \
#            .addGrid(als.maxIter, [5, 50, 100, 200]) \
#            .addGrid(als.regParam, [.01, .05, .1, .15]) \
#            .build()

# Define evaluator as RMSE and print length of evaluator
#evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
#print ("Num models to be tested: ", len(param_grid))

In [0]:
# Build cross validator
#cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
#print (cv)


In [0]:
#Fit cross validator to the 'train' dataset

#model = cv.fit(train)

#Extract best model from the cv model above

#best_model = model.bestModel


Best Model and Best Model Parameters

In [0]:
# Print best_model

#print(type(best_model))

# Complete the code below to extract the ALS model parameters

#print("**Best Model**")

# Print "Rank"

#print("  Rank:", best_model.getRank())

# Print "MaxIter"

#print("  MaxIter:", best_model.getMaxIter())

# Print "RegParam"

#print("  RegParam:", best_model.getRegParam())

Generate predictions and calculate RMSE

In [0]:
  # View the predictions 
#test_predictions.show()

# Calculate and print the RMSE of the test_predictions

#RMSE = evaluator.evaluate(test_predictions)
#print (RMSE)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

# Complete the evaluator code
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Extract the 3 parameters
print (evaluator.getMetricName())
print (evaluator.getLabelCol())
print (evaluator.getPredictionCol())

# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print (RMSE)

test_predictions.show()

rmse
rating
prediction
0.9090895910523555
+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   232|    463|   4.0| 3.4895976|
|   452|    463|   2.0| 2.8818955|
|   534|    463|   4.0|   3.36804|
|    86|    471|   4.0| 4.1151204|
|   292|    471|   3.5| 4.0617824|
|   306|    471|   3.0| 3.7516801|
|   491|    471|   3.0| 4.0695353|
|   309|    471|   4.0|  4.269315|
|   607|    471|   4.0| 3.7564955|
|    23|    471|   3.5| 3.8811774|
|   514|    471|   4.0|   4.18469|
|   585|    471|   4.0|  4.431021|
|   574|    471|   3.5| 3.8619456|
|   354|    471|   5.0| 4.1767726|
|   242|    471|   5.0| 4.4315233|
|   529|    471|   4.0| 3.5643702|
|    30|    471|   4.0| 4.1113615|
|   311|    471|   0.5|  2.540586|
|   399|    471|   5.0| 2.9007144|
|   509|    496|   3.0| 2.1143327|
+------+-------+------+----------+
only showing top 20 rows

